## EVALUATION

In [ ]:
# Evaluattion
!CUDA_VISIBLE_DEVICES='0' python ./src/eval_custom.py \
--load=train_log/best_config/export/max-eval-mAP \
--config=configs/best_config.json \
--output_dir=train_log/best_config/eval_result.json \

# MODEL LOAD

In [ ]:
# PACKAGE LOAD AND PATH SETTING

import os
import sys

os.environ["CUDA_VISIBLE_DEVICES"]="0" # GPU 필요
sys.path.append(os.path.abspath('./src'))

from glob import glob
import pandas as pd

import argparse
import itertools
import numpy as np
import shutil
import tensorflow as tf
import cv2
import tqdm

import matplotlib.pyplot as plt 

import tensorpack.utils.viz as tpviz
from tensorpack.predict import MultiTowerOfflinePredictor, OfflinePredictor, PredictConfig
from tensorpack.tfutils import SmartInit, get_tf_version_tuple
from tensorpack.tfutils.export import ModelExporter
from tensorpack.utils import fs, logger
from tensorpack.tfutils import get_model_loader

from dataset import DatasetRegistry, register_custom_data
from config import config as cfg
from config import finalize_configs
from data import get_eval_dataflow, get_train_dataflow
from eval import DetectionResult, multithread_predict_dataflow, predict_image
from modeling.generalized_rcnn import ResNetC4Model, ResNetFPNModel
from viz import (
    draw_annotation, draw_final_outputs, draw_predictions,
    draw_proposal_recall, draw_final_outputs_blackwhite)


# PATH 로드
CONFIG_PATH = './configs/best_config.json'
IMAGE_PATH =''

MODEL_PATH = './train_log/best_config/export/max-eval-mAP'

anno = pd.read_csv('./custom_data/annotation.csv')
eval_images = list(anno[anno['is_train']==False]['img_path'].unique())

In [ ]:
# Config Setting AND Model Load
cfg.update_from_json(CONFIG_PATH)
cfg.DATA.CLASS_NAMES = ['BG'] + cfg.DATA.CLASS_NAMES
    
register_custom_data()
finalize_configs(is_training=False)

model = ResNetFPNModel() if cfg.MODE_FPN else ResNetC4Model()

# cfg.TEST.RESULT_SCORE_THRESH = cfg.TEST.RESULT_SCORE_THRESH_VIS

logger.info("Loading checkpoint from {}".format(MODEL_PATH))
predcfg = PredictConfig(model=model,
                        session_init=get_model_loader(MODEL_PATH),
                        input_names=model.get_inference_tensor_names()[0],
                        output_names=model.get_inference_tensor_names()[1])

pred_func = OfflinePredictor(predcfg)

## PRECISON RECALL INFO

In [ ]:
def get_IoU(boxA, boxB):
    x1 = max(boxA[0], boxB[0])
    y1 = max(boxA[1], boxB[1])
    x2 = min(boxA[2], boxB[2])
    y2 = min(boxA[3], boxB[3])
    
    width = (x2-x1)
    height = (y2-y1)
    
    if width < 0 or height <0:
        return 0.0
    
    inter_area = max(0, x2 - x1 +1) * max(0, y2- y1 +1)
    
    boxA_area = (boxA[2] - boxA[0] +1) * (boxA[3] - boxA[1] +1)
    boxB_area = (boxB[2] - boxB[0] +1) * (boxB[3] - boxB[1] +1)
    
    iou = inter_area / float(boxA_area + boxB_area - inter_area)
    
    return iou

In [ ]:
# 이미지 별 검출 객체 비교

import pandas as pd
from tqdm import tqdm

save_path = './recall_precision.csv'
pred_info = pd.DataFrame(columns=['img_path','target_class','n_target','n_pred','n_true','n_false'])

THRE = 0.5

for idx, input_file in tqdm(enumerate(eval_images)):
    
    img = cv2.imread(input_file, cv2.IMREAD_COLOR)
    
    img_name = input_file.split('/')[-1]
    target_class = None
    n_target = None
    n_pred = None
    n_true = 0
    n_false = 0

    # GT 정보
    gt_boxes = anno[anno['img_path']==input_file][['x1','y1','x2','y2']].values
    gt_labels = anno[anno['img_path']==input_file]["category_id"].values
    
    n_target = len(gt_labels)
    target_class = gt_labels[0]
    target_name = anno[anno['img_path']==input_file]["label"].values[0]
    
    # Pred 정보
    results = predict_image(img, pred_func)
    n_pred = len(results)
    
    # True / False 계산
    for result in results:
        pred_box, _, pred_label, _ = result
        
        max_iou = 0
        max_iou_idx = -1
        
        for gt_idx, gt_box in enumerate(gt_boxes):
            iou = get_IoU(gt_box, pred_box)
            
            if iou > max_iou:
                max_iou = iou
                max_iou_idx = gt_idx
        
        if (max_iou > THRE) and (pred_label == target_class) : n_true+=1
        else: n_false+=1
    
    pred_info.loc[idx] = [img_name, target_name, n_target, n_pred, n_true, n_false]
pred_info.to_csv('./report/faster_rcnn_PR.csv')

## IMAGE SAVE

In [ ]:
# IMAGE SAVE
from tqdm import tqdm
save_path = './report/save_image/'
for input_file in tqdm(eval_images):
    img_name = input_file.split('/')[-1]
    img = cv2.imread(input_file, cv2.IMREAD_COLOR)

    # GT 이미지 그리기
    gt_img = img.copy()
    for x1, y1, x2, y2 in anno[anno['img_path']==input_file][['x1','y1','x2','y2']].values:
        gt_img = cv2.rectangle(gt_img,(int(x1),int(y1)),(int(x2),int(y2)),(255,0,0),1)
        
    # Pred 이미지 그리기    
    results = predict_image(img, pred_func)
    pred_img = draw_final_outputs(img, results)

    save_img = np.concatenate([gt_img,pred_img],axis=1)
    cv2.imwrite(save_path+img_name,cv2.cvtColor(save_img,cv2.COLOR_BGR2RGB))

## VISUALIZATION

In [ ]:
# Predict And Show
# 수정
for input_file in eval_images[0:20]:
    print(input_file)
    img = cv2.imread(input_file, cv2.IMREAD_COLOR)

    # GT 이미지 그리기
    gt_img = img.copy()
    for x1, y1, x2, y2 in anno[anno['img_path']==input_file][['x1','y1','x2','y2']].values:
        gt_img = cv2.rectangle(gt_img,(int(x1),int(y1)),(int(x2),int(y2)),(255,0,0),1)
        
    # Pred 이미지 그리기    
    results = predict_image(img, pred_func)
    pred_img = draw_final_outputs(img, results)

    fig = plt.figure(figsize=(15,15))
    
    ax1 = fig.add_subplot(1,2,1)
    ax1.imshow(gt_img)
    ax1.set_title("GT IMAGE")
    ax1.axis("off")
    
    ax2 = fig.add_subplot(1,2,2)
    ax2.imshow(pred_img)
    ax2.set_title("PRED IMAGE")
    ax2.axis("off")
    
    plt.show()
    plt.close()
    

## Category 별 COCO AP 계산

In [ ]:
from pycocotools.cocoeval import COCOeval
from pycocotools.coco import COCO
import json
from tqdm import tqdm

EVAL_ANNO_PATH = './custom_data/annotations/eval.json'
SAVE_PATH = './report/AP/'

In [ ]:
# 객체별 검출결과 json 생성
image_ids = {}
with open(EVAL_ANNO_PATH) as f : gt = json.load(f)
    
for eval_info in gt['images']:
    image_ids[eval_info['file_name']] = eval_info['id']

write_json_list = [[],[],[],[],[]]
total_write_json = []

pred_image_id = [[],[],[],[],[]]

for idx, input_file in tqdm(enumerate(eval_images)):
    img = cv2.imread(input_file, cv2.IMREAD_COLOR)
    
    img_name = input_file.split('/')[-1]
    img_id = image_ids[img_name]
    
    results = predict_image(img, pred_func)
    
    for result in results:
        box, score, category_id, _ = result
        
        box = list(box)
        box = [float(b) for b in box]
        width, height = box[2]-box[0], box[3]-box[1]
        box[2],box[3] = width, height
        
        data = {'image_id': img_id,
                'category_id': category_id,
                'score': float(score),
                'bbox': box}
        pred_image_id[category_id-1].append(img_id)
        
        write_json_list[category_id-1].append(data)
        total_write_json.append(data)

pred_image_id = [list(set(ids)) for ids in pred_image_id]

for idx,write_json in enumerate(write_json_list):
    with open(SAVE_PATH+str(idx+1)+'_detection.json','w') as f : json.dump(write_json,f)
        
with open(SAVE_PATH+'all_detection.json','w') as f : json.dump(total_write_json,f)

In [ ]:
with open(EVAL_ANNO_PATH,'r') as f : gt = json.load(f)
with open(SAVE_PATH+'all_gt.json','w') as f : json.dump(gt,f)
    
data ={
    'images' : [],
    'categories' : [],
    'annotations' : []
}

image_id_list = [[],[],[],[],[]]
# data['categories'] = gt['categories']

# Annotation 나누기
for anno_info in gt['annotations']:
    category_id = anno_info['category_id']
    image_id_list[category_id-1].append(anno_info['image_id'])

# 이미지 ID 별 리스트
# gt 이미지 리스트
image_id_list = [list(set(ids)) for ids in image_id_list]

# pred에서 검출된 image id 리스트
ids=[]
for i in range(5):
    ids.append(image_id_list[i]+pred_image_id[i])
    
merge_image_ids = [list(set(img_id)) for img_id in ids]

for idx, image_ids in enumerate(merge_image_ids):
    
    write_data ={
        'images' : [],
        'categories' : [],
        'annotations' : []}

    write_data['categories'] = gt['categories']
    
    for i_id in image_ids:
        
        for image_info in gt['images']:
            if image_info['id'] == i_id:
                write_data['images'].append(image_info)
                
        for anno_info in gt['annotations']:
            if anno_info['image_id']==i_id:
                
                if (i_id not in image_id_list[idx]) or (anno_info['category_id'] != idx+1)  : 
                    continue
                
                else: 
                    write_data['annotations'].append(anno_info)
            
    with open(SAVE_PATH+str(idx+1)+'_gt.json','w') as f : json.dump(write_data,f) 

In [ ]:
#객체별 COCO AP 계산

class_names = ['ALL','3','10','13','16','17']
ap_labels = pd.DataFrame(columns=['AP(IoU=0.50)','AP(IoU=0.75)','AP(IoU=0.50:0.95)','AP(Small)', 'AP(Medium)', 'AP(Large)'],index=class_names)

for i in range(6):
    target_class = str(i)
    gt_path = SAVE_PATH+target_class+'_gt.json'
    detection_path = SAVE_PATH+target_class+'_detection.json'
    
    if i==0:
        target_class = 'all'
        gt_path = './report/AP/all_gt.json'
        detection_path = './report/AP/all_detection.json'
        
    coco_true = COCO(gt_path)
    coco_pred = coco_true.loadRes(detection_path)
    
    coco_eval = COCOeval(coco_true, coco_pred, 'bbox')    
    print ("<<{} mAP>>".format(target_class))
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()
    print("\n\n\n\n\n")
    
    result = coco_eval.stats
    ap_labels.iloc[i] = [result[1],result[2],result[0],result[3],result[4],result[5]]
    
ap_labels.to_csv('./report/AP/coco_ap_result.csv')
ap_labels